<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import" data-toc-modified-id="Import-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import</a></span></li><li><span><a href="#Compile" data-toc-modified-id="Compile-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Compile</a></span></li><li><span><a href="#Link" data-toc-modified-id="Link-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Link</a></span><ul class="toc-item"><li><span><a href="#Define-struct" data-toc-modified-id="Define-struct-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Define struct</a></span></li><li><span><a href="#Link" data-toc-modified-id="Link-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Link</a></span></li><li><span><a href="#Wrappers" data-toc-modified-id="Wrappers-3.3"><span class="toc-item-num">3.3&nbsp;&nbsp;</span>Wrappers</a></span></li></ul></li><li><span><a href="#Run" data-toc-modified-id="Run-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Run</a></span></li><li><span><a href="#Memory" data-toc-modified-id="Memory-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Memory</a></span></li><li><span><a href="#NLopt" data-toc-modified-id="NLopt-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>NLopt</a></span></li><li><span><a href="#OpenMP" data-toc-modified-id="OpenMP-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>OpenMP</a></span></li></ul></div>

# Import

In [1]:
# magics and imports
%matplotlib inline
%load_ext autoreload
%autoreload 1

# a. external
import os
import time
import copy
import ctypes as ct
from types import SimpleNamespace
import numpy as np
from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10, 'font.family': 'STIXGeneral', 'mathtext.fontset': 'stix'})

# b. internal
import converter

In [2]:
%%html
<style>
.output_wrapper, .output {
    height:auto !important;
    max-height:5000px;  /* your desired max-height here */
}
.output_scroll {
    box-shadow:none !important;
    webkit-box-shadow:none !important;
}
</style>

In [3]:
def print_file(filename):
    with open(filename, 'r') as f:
        print(f.read())

In [4]:
def clean():
    dir_name = os.getcwd()
    test = os.listdir(dir_name)
    for item in test:
        exts = ['.exe','.lib','.exp','.obj']
        for ext in exts:
            if item.endswith(ext):
                os.remove(os.path.join(dir_name, item))

# Compile

Set libraries to none:

In [5]:
msvs_lib = None
intel_lib = None

Define delinker:

In [6]:
def delink_cpp(mylib):
    
    if mylib == None:
        return mylib

    # 1. get handle
    handle = mylib._handle
    
    # 2. delete linking variable
    del mylib
    mylib = None

    # 3. free handle
    ct.windll.kernel32.FreeLibrary.argtypes = [ct.wintypes.HMODULE]
    ct.windll.kernel32.FreeLibrary(handle)
    
    print('delinked')
    
    return mylib

**Compile with MSVC**

In [7]:
msvs_lib = delink_cpp(msvs_lib)
filename = 'compile_msvc.bat'
print_file(filename)
os.system(filename) # produce .dll file

# clean up
clean()

cd "C:\Program Files (x86)\Microsoft Visual Studio\2017\Community\VC\Auxiliary\Build\"
call vcvarsall.bat x64
cd "C:\Users\okoJD\Dropbox\CodeExamples\ctypes"
cl /LD /Ox test.cpp


**Compile with Intel**

In [8]:
intel_lib = delink_cpp(intel_lib)
filename = 'compile_intel.bat'
print_file(filename)
os.system(filename) # produce .dll file

# clean up
clean()

call "C:\Program Files (x86)\IntelSWTools\compilers_and_libraries_2017.6.270\windows\bin\ipsxe-comp-vars.bat" intel64 vs2017
icl /O3 test.cpp
xilink -nologo -dll -out:test_intel.dll test.obj


# Link

## Define struct

Create a Python version of the C++ struct:

In [9]:
class par_struct(ct.Structure):
    filename = 'test.cpp'
    struct_name = 'par_struct'
    var_type_str_list = ['int','double']
    _fields_ = converter.get_fields(filename,struct_name,var_type_str_list)

Print the contents:

In [10]:
for field in par_struct._fields_:
    print(field)

('n', <class 'ctypes.c_long'>)
('a', <class 'converter.LP_c_double'>)
('b', <class 'converter.LP_c_double'>)
('c', <class 'converter.LP_c_double'>)


## Link

In [11]:
# a. link
msvc_lib = ct.cdll.LoadLibrary('test.dll') # load .dll file

# b. result and argument types
msvc_lib.square_array.restype = None     
msvc_lib.square_array.argtypes = [ct.POINTER(par_struct)]

In [12]:
print(msvc_lib)

<CDLL 'test.dll', handle 7ffbf8140000 at 0x1ecd28667b8>


In [13]:
# a. link
intel_lib = ct.cdll.LoadLibrary('test_intel.dll') # load .dll file

# b. result and argument types
intel_lib.square_array.restype = None     
intel_lib.square_array.argtypes = [ct.POINTER(par_struct)]

In [14]:
print(intel_lib)

<CDLL 'test_intel.dll', handle 7ffbf70f0000 at 0x1ecd2866a58>


## Wrappers

In [15]:
# c. wrapper     
def square_array(par, intel=False):
    
    p_par = converter.get_pointers(par,par_struct)    
    if intel:
        msvc_lib.square_array(ct.byref(p_par))
    else:
        intel_lib.square_array(ct.byref(p_par))
                    
    # access newly allocated memory
    par.c = np.ctypeslib.as_array(p_par.c, shape=(par.n,))
    
    # d. print
    print('a:')
    for i in range(par.n):
        print(' {:2d}{:8.0f}'.format(i,par.a[i]))

    print('b:')
    for i in range(par.n):
        print(' {:2d}{:8.0f}'.format(i,par.b[i]))

    print('c:')
    for i in range(par.n):
        print(' {:2d}{:8.0f}'.format(i,par.c[i]))   
        
def clear_mem(par, intel=False):
    
    p_par = converter.get_pointers(par,par_struct)    
    if intel:
        msvc_lib.clear_mem(ct.byref(p_par))
    else:
        intel_lib.clear_mem(ct.byref(p_par))

# Run

In [16]:
# a. define
par = dict()
par = SimpleNamespace(**par)

# b. set values
par.n = 5
par.a = np.linspace(1.0,20.0,par.n,dtype=np.float64)
par.b = np.zeros(par.n,dtype=np.float64)
par.c = np.zeros(0,dtype=np.float64)

par_intel = copy.deepcopy(par)

# c. msvc
print('msvc')
square_array(par)

print('')
print('intel')
square_array(par_intel,intel=True)

msvc
a:
  0       1
  1       6
  2      10
  3      15
  4      20
b:
  0       1
  1      33
  2     110
  3     233
  4     400
c:
  0      11
  1      43
  2     120
  3     243
  4     410

intel
a:
  0       1
  1       6
  2      10
  3      15
  4      20
b:
  0       1
  1      33
  2     110
  3     233
  4     400
c:
  0      11
  1      43
  2     120
  3     243
  4     410


# Memory

The memory of par.c is the memory we originally allocated in C++.

In [17]:
print(par.c)
clear_mem(par)
time.sleep(2)
print(par.c)

[ 11.      43.0625 120.25   242.5625 410.    ]
[ 11.      43.0625 120.25   242.5625 410.    ]


# NLopt

In [18]:
# a. compile
filename = 'compile_intel_nlopt.bat'
os.system(filename)
clean()
          
# b. link
intel_lib_nlopt = ct.cdll.LoadLibrary('test_intel_nlopt.dll') # load .dll file

# c. result and argument types
intel_lib_nlopt.optimize.restype = None     
intel_lib_nlopt.optimize.argtypes = None

# d. run
intel_lib_nlopt.optimize()
print_file('log_nlopt.txt')

# e. deling
intel_lib_nlopt = delink_cpp(intel_lib_nlopt)

found minimum at f(0.21825,0.38175) = 0.03134113069
time:  0.30, inside  0.30, evals = 443
delinked


# OpenMP

In [19]:
# a. compile
filename = 'compile_intel_openmp.bat'
os.system(filename)
clean()

# b. link
intel_lib_openmp = ct.cdll.LoadLibrary('test_intel_openmp.dll') # load .dll file

# c. result and argument types
intel_lib_openmp.parallize.restype = None     
intel_lib_openmp.parallize.argtypes = None

# d. run
intel_lib_openmp.parallize()
print_file('log_openmp.txt')

# e. deling
intel_lib_openmp = delink_cpp(intel_lib_openmp)

task = (3,0) handled by thread = 3
task = (2,0) handled by thread = 2
task = (1,0) handled by thread = 1
task = (0,0) handled by thread = 0
task = (3,1) handled by thread = 3
task = (2,1) handled by thread = 2
task = (1,1) handled by thread = 1
task = (0,1) handled by thread = 0
task = (3,2) handled by thread = 3
task = (2,2) handled by thread = 2
task = (1,2) handled by thread = 1
task = (0,2) handled by thread = 0
task = (3,3) handled by thread = 3
task = (2,3) handled by thread = 2
task = (1,3) handled by thread = 1
task = (0,3) handled by thread = 0

delinked
